In [35]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
filelist = os.listdir("seoul")[1:]

os를 통해 전체 파일을 불러옵니다. 첫 번째 파일을 제외한 이유는 이후 for문을 돌릴 때 가장 처음 데이터프레임이 있어야 이후에 파일을 붙일 수 있기 때문입니다. 먼저 가장 위의 파일인 강남구 갑 타겟 데이터를 전처리 하고 for문을 돌리도록 하겠습니다.

In [37]:
gangnamgap = pd.read_excel("C:/Users/lhmlh/degong/seoul/개표상황(투표구별)_강남구갑.xlsx")
header = gangnamgap.iloc[2,:4]
header

개표상황(투표구별)    읍면동명
Unnamed: 1    투표구명
Unnamed: 2    선거인수
Unnamed: 3     투표수
Name: 2, dtype: object

강남구 갑 타켓데이터 파일을 불러옵니다. 어떤 부분을 남길지를 대략적인 데이터 형태 탐색해봤습니다.

In [38]:
a = pd.Series(gangnamgap.iloc[3,4:].values)
a

0       새누리당\n이종구
1     더불어민주당\n김성곤
2              \n
3              \n
4              \n
5              \n
6              \n
7              \n
8              \n
9              \n
10             \n
11             \n
12             \n
13             \n
14             \n
15             \n
16             \n
17             \n
18             \n
19             \n
20             \n
21              계
22            NaN
23            NaN
24            NaN
dtype: object

In [39]:
a = a.apply(lambda x: str(x).split("\n")[0])
a

0       새누리당
1     더불어민주당
2           
3           
4           
5           
6           
7           
8           
9           
10          
11          
12          
13          
14          
15          
16          
17          
18          
19          
20          
21         계
22       nan
23       nan
24       nan
dtype: object

In [40]:
a = a[(a!="")&(a!="nan")]
a

0       새누리당
1     더불어민주당
21         계
dtype: object

In [41]:
gangnamgap = gangnamgap[gangnamgap['Unnamed: 1']=="소계"]
gangnamgap.head()

,개표상황(투표구별),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
9,신사동,소계,14827,7771,4774,2817,0,0,0,0,...,0,0,0,0,0,NaN,7591,180,7056,NaN
16,논현1동,소계,21876,8939,3966,4769,0,0,0,0,...,0,0,0,0,0,NaN,8735,204,12937,NaN
23,논현2동,소계,19063,8887,4639,4017,0,0,0,0,...,0,0,0,0,0,NaN,8656,231,10176,NaN
30,압구정동,소계,22435,12118,8301,3539,0,0,0,0,...,0,0,0,0,0,NaN,11840,278,10317,NaN
39,청담동,소계,24507,12470,7366,4777,0,0,0,0,...,0,0,0,0,0,NaN,12143,327,12037,NaN


데이터를 살펴보면 저희가 분석에 필요한 부분은 동별 전체 투표수이기 때문에 투표소별 투표수는 전부 삭제하고 동별 총 투표 소계만 남겼습니다.

In [42]:
gangnamgap = gangnamgap.iloc[:,:len(a)+3]
gangnamgap.columns = np.append(header,a[:-1])
gangnamgap.head()

,읍면동명,투표구명,선거인수,투표수,새누리당,더불어민주당
9,신사동,소계,14827,7771,4774,2817
16,논현1동,소계,21876,8939,3966,4769
23,논현2동,소계,19063,8887,4639,4017
30,압구정동,소계,22435,12118,8301,3539
39,청담동,소계,24507,12470,7366,4777


강남구갑에 대한 전처리를 마쳤습니다

In [43]:
first = gangnamgap
gangnamgap['자치구'] = "강남구"
for file in filelist:
    data=pd.read_excel(f'seoul/{file}')
    a=pd.Series(data.iloc[3,4:].values)
    a = a.apply(lambda x: str(x).split("\n")[0])
    a = a[(a!="")&(a!="nan")]
    data = data[data['Unnamed: 1']=="소계"]
    data = data.iloc[:,:len(a)+3]
    data.columns = np.append(header,a[:-1])
    gu = re.findall("_\w+구", file)[0][1:]
    data['자치구'] = gu
    first = first.append(data, sort=False)

다른 구의 데이터 형태가 거의 동일하기 때문에 강남구 갑에 대해 전처리 한 결과를 일반화하여 for 문을 돌리고 append를 통해 이어 붙였습니다.
중간에 자치구를 굳이 붙인 이유는 서울에 신사동이 2개가 있어서 구분하지 않으면 전처리에 문제가 생기는 관계로 같이 넣었습니다.

In [44]:
Second = first[["읍면동명","자치구", "선거인수", "새누리당", "더불어민주당", "국민의당"]]
Second.읍면동명=Second.읍면동명.apply(lambda x: re.sub(r'제', '', str(x)))
Second.읍면동명=Second.읍면동명.apply(lambda x: re.sub(r'·', '.', str(x)))
Second.head()

,읍면동명,자치구,선거인수,새누리당,더불어민주당,국민의당
9,신사동,강남구,14827,4774,2817,NaN
16,논현1동,강남구,21876,3966,4769,NaN
23,논현2동,강남구,19063,4639,4017,NaN
30,압구정동,강남구,22435,8301,3539,NaN
39,청담동,강남구,24507,7366,4777,NaN


Merge를 하는 과정에서 읍면동명이 독립변수 데이터와 타켓변수 데이터 간에 다른 점이 있어 통일하는 작업을 거쳤습니다. 
EX) 논현제1동 -> 논현1동, 종로 1·2·3·4동 -> 종로 1.2.3.4동

In [45]:
old_man = pd.read_csv("C:/Users/lhmlh/degong/Data_GongHak/data/독립변수_데이터_호식/서울시 독거노인 현황 (성별,동별) 통계_2016.txt", engine="python" ,encoding="utf-8", sep="\t")
old_man.head()

,기간,자치구,동,합계,합계.1,합계.2,국민기초생활보장수급권자,국민기초생활보장수급권자.1,국민기초생활보장수급권자.2,저소득노인,저소득노인.1,저소득노인.2,일반,일반.1,일반.2
0,기간,자치구,동,계,남,여,계,남,여,계,남,여,계,남,여
1,2016,합계,합계,"288,599",…,…,"51,528",…,…,"16,104",…,…,"220,967",…,…
2,2016,종로구,소계,"7,074","2,811","4,263","1,189",584,605,277,97,180,"5,608","2,130","3,478"
3,2016,종로구,사직동,383,132,251,38,12,26,12,7,5,333,113,220
4,2016,종로구,삼청동,156,49,107,17,5,12,4,-,4,135,44,91


In [46]:
old_man = old_man[["동", "자치구", "합계"]]
old_man=old_man.rename(columns={"동": "읍면동명"})
old_man = old_man.drop([0,1,2])
old_man.head()

,읍면동명,자치구,합계
3,사직동,종로구,383
4,삼청동,종로구,156
5,부암동,종로구,319
6,평창동,종로구,615
7,무악동,종로구,205


In [47]:
old_man.자치구=old_man.자치구.apply(lambda x: re.sub(r'성동', '중', str(x)))
old_man[old_man.자치구=="중구"]

,읍면동명,자치구,합계
20,소계,중구,"5,528"
21,소공동,중구,54
22,회현동,중구,461
23,명동,중구,157
24,필동,중구,228
25,장충동,중구,187
26,광희동,중구,352
27,을지로동,중구,224
28,신당5동,중구,411
29,황학동,중구,514


타켓 데이터의 자치구가 중구와 성동구가 같이 묶여있는데 반해 독립변수 데이터의 자치구는 중구와 성동구가 분리되어 있어 통일시키기 위해 성동구로 되어 있는 독립변수 데이터의 자치구를 모두 중구로 변경

In [48]:
second_old_man = pd.merge(Second, old_man, how='left')
second_old_man = second_old_man.rename(columns={"합계":"독거노인수"})
second_old_man.head()

,읍면동명,자치구,선거인수,새누리당,더불어민주당,국민의당,독거노인수
0,신사동,강남구,14827,4774,2817,NaN,380
1,논현1동,강남구,21876,3966,4769,NaN,500
2,논현2동,강남구,19063,4639,4017,NaN,409
3,압구정동,강남구,22435,8301,3539,NaN,647
4,청담동,강남구,24507,7366,4777,NaN,491


In [49]:
buyang = pd.read_csv("C:/Users/lhmlh/degong/Data_GongHak/data/독립변수_데이터_호식/서울시 부양비 및 노령화지수 (동별) 통계_2016.txt", engine="python" ,encoding="utf-8", sep="\t")
buyang.head()

,기간,자치구,동,유년부양비,노년부양비,노령화지수
0,2016,합계,합계,15.2,16.8,110.8
1,2016,종로구,소계,12.2,20.6,168.2
2,2016,종로구,사직동,16.5,24.0,144.8
3,2016,종로구,삼청동,12.7,28.2,221.3
4,2016,종로구,부암동,13.4,21.0,156.6


In [50]:
buyang = buyang.rename(columns={"동":"읍면동명"})
buyang = buyang[~buyang.읍면동명.isin(["소계", "합계"])]
buyang=buyang.drop("기간", axis=1)
buyang.head()

,자치구,읍면동명,유년부양비,노년부양비,노령화지수
2,종로구,사직동,16.5,24.0,144.8
3,종로구,삼청동,12.7,28.2,221.3
4,종로구,부암동,13.4,21.0,156.6
5,종로구,평창동,14.8,20.5,138.7
6,종로구,무악동,20.3,19.8,97.9


In [54]:
buyang.자치구=buyang.자치구.apply(lambda x: re.sub(r'성동', '중', str(x)))
buyang[buyang.자치구=="중구"]

,자치구,읍면동명,유년부양비,노년부양비,노령화지수
20,중구,소공동,15.6,11.2,71.6
21,중구,회현동,8.5,29.7,350.0
22,중구,명동,8.5,24.6,288.1
23,중구,필동,9.1,19.6,215.5
24,중구,장충동,9.1,16.2,178.7
25,중구,광희동,8.1,23.2,285.3
26,중구,을지로동,5.8,31.9,549.4
27,중구,신당5동,13.1,19.2,146.8
28,중구,황학동,11.6,16.7,143.6
29,중구,중림동,13.2,23.7,179.6


In [56]:
old_man_buyang=pd.merge(second_old_man, buyang, how='left')
old_man_buyang.head()

,읍면동명,자치구,선거인수,새누리당,더불어민주당,국민의당,독거노인수,유년부양비,노년부양비,노령화지수
0,신사동,강남구,14827,4774,2817,NaN,380,15.8,19.1,121.3
1,논현1동,강남구,21876,3966,4769,NaN,500,7.2,12.4,172.1
2,논현2동,강남구,19063,4639,4017,NaN,409,11.9,14.9,124.8
3,압구정동,강남구,22435,8301,3539,NaN,647,20.0,21.4,107.1
4,청담동,강남구,24507,7366,4777,NaN,491,16.3,15.2,93.5
